# Scripting with MetSim

This notebook provides a tutorial for running MetSim in a scripting environment. 
The scripting environment allows for a more interactive workflow that will make it easier to manipulate the data that MetSim generates.

Then, we will begin to expand on this to show how MetSim can be used programmatically to generate multiple sets of meteorologic data.
We will run multiple longwave radiation parameterizations and compare their relative qualities.
Finally, we will look at how changing some of the various parameters which MetSim exposes affect the output.

First, we begin with some standard imports. For this portion of the tutorial we will be making use of several libraries for analysis and plotting:

 - `xarray` is used under the hood for managing the output from MetSim
 - `pandas` is used for setting up the dates for our MetSim run
 - `holoviews` and `geoviews` will be used for some interactive plotting
 - `geopandas` is used for used for managing the shapefiles, also used for plotting
 - `cartopy` is also used for managing the shapefiles, also used for plotting
 - `metsim`, obviously, will be used for data processing

We initialize the notebook by importing these, as well as the `pylab` cell magic, which imports `numpy` and `matplotlib`. 
We also set up `holoviews` to use the `bokeh` interactive plotting library as the backend.

Note that there may be some warnings from various Python libraries throughout this tutorial which are not issues as of the presentation date.

In [ ]:
%pylab inline
import cartopy
import geoviews as gv
import geopandas as gpd
import holoviews as hv
import pandas as pd
import xarray as xr

from metsim import MetSim
hv.notebook_extension('bokeh')

# Our study location: the Green River basin

For this example we will be showing how MetSim can be used on unstructured mesh arrangements as well as the structured latitude-longitude grid that we ran in the command line examples.
We will use a setup which consists of twelve "hydrologic response units" or HRU.
A HRU typically delineates a watershed by topography, soil type, land use, or other defining feature.
The HRU delineation we are using comes from the USGS hydrologic unit codes (HUC), with 12 digit codes. 
Commonly this is referred to as HUC12, which is the finest resolution.
Coarser resolutions are available as HUC10 and HUC8.

The 12 HRU that we will be running are located along the Green river, just southeast of Seattle. 
It originates on the western slopes of the Cascade mountains.
The upper portion of the Green river provides much of the water for the city of Tacoma, which is south of Seattle.
The Green river ultimately ends up as a tributary to the Duwamish river, which flows through Seattle and into Puget Sound.

Let's take a look. 
Because this is an unstructured mesh we will be defining the mesh elements by their respective HUC12 identifiers. 
We can draw boundaries using a shapefile, which we have obtained from the USGS datasets and included in the tutorial data.
There are several tools for plotting maps in Python using shapefiles, most notably Cartopy, GeoPandas, and GeoViews.
Other tools for more general GIS functionality include GDAL, Rasterio, Shapely, and Fiona.
We will be using Cartopy and GeoPandas to define the geometries and then a combination of GeoViews and HoloViews to actually draw the plots.

In [ ]:
shapefile = './shapefile/shapefile.shp'
shapes = cartopy.io.shapereader.Reader(shapefile)
(gv.tile_sources.StamenTerrainRetina
 * gv.Shape.from_shapefile(shapefile, crs=cartopy.crs.PlateCarree()).opts(
     style=dict(fill_color='honeydew', line_color='darkolivegreen', alpha=0.8))
 * gv.Points([(-122.3321, 47.9),(-122.8443, 47.2529), (-122.5, 47.0)]).opts(style=dict(size=0))
).opts(width=900, height=500)

# Getting set up: Parameters and MetSim instantiation

Unlike the command line usage of MetSim, while in the scripting enviornment we will provide the confiugration as a Python dictionary. 
The information that this dictionary, which we will call `params`, is similar to that which is found in the configuration file.
However, there are some notable differences which we will walk through.

First, note that we must provide dates as `datetime` objects. 
This is easily handled via `pandas`. 
Second, we point out a new parameter, the `scheduler`; we have set it to `threading` for a specific purpose.
As mentioned in the command line portion of the tutorial, the scheduler is how MetSim manages jobs under the hood.
The default value is the `distributed` scheduler, which does not currently support running in this sort of interactive environment.
By setting `scheduler` to `threading` we are able to work in this environment.
A notable drawback to this, is that parallelism is significantly hindered. 
We hope that future versions of `dask` (the library which we use as a job manager) improve this compatibility, and we can make it available in MetSim.


In [ ]:
# rest of the workflow
dates = pd.date_range('1971/01/01', '1971/12/31')
params = {
    # Run parameters, timestep, start, and stop dates
    'time_step'    : "60",       
    'start'        : dates[0],
    'stop'         : dates[-1],
    # Where all of our input files are
    'forcing'      : './input/green_river_forcing.nc',     
    'domain'       : './input/green_river_domain.nc',
    'state'        : './input/green_river_state.nc',
    # The format of the forcing data
    'forcing_fmt'  : 'netcdf',
    # Specification of the output
    'out_dir'      : './output',
    'out_prefix': 'green_river',
    # Scheduler to use for job management - see text above for more information
    'scheduler'    : 'threading',
    # This is analagous to the [chunks] section from the config file
    'chunks'       : 
        {'hru': 3},
    # This is analagous to the [forcing_vars] section from the config file
    'forcing_vars' : 
        {'prcp' : 'prec', 'tmax': 't_max', 'tmin': 't_min'},
    # This is analagous to the [state_vars] section from the config file
    'state_vars'   : 
        {'prcp' : 'prec', 'tmax': 't_max', 'tmin': 't_min'},
    # This is analagous to the [domain_vars] section from the config file
    'domain_vars'  : 
        {'elevation': 'elev', 'latitude': 'lat', 'longitude': 'lon', 'mask': 'mask'}
    }               

# Okay, let's run MetSim!

## First, let's create our MetSim object
For this step, we simply give the parameters as an argument to the MetSim object

In [ ]:
ms = MetSim(params)

## Now, we can run the simulation
This is as easy as calling the object's `run` method. Try it!

In [ ]:
ms.run()

## Finally, let's open up the output and see what we've got
Here we can see that we've got a dataset that looks like we would expect. 
It's got the twelve HRU each with 8760 timesteps (equal to 365 * 24).
We also have 6 variables:
 - temperature
 - precipitation
 - shortwave
 - longwave
 - vapor pressure
 - relative humidity
 
The output is simply an `xarray` dataset, which was read in from the underlying NetCDF file that the `ms.run()` command generated.
You can do all of the standard things that were taught in the cyberseminars.
Let's start by just printing out what's there like before.

In [ ]:
out = ms.open_output().load()
out = out.sortby('hru')
gdf = gpd.read_file(shapefile)
print(out)

# Interacting with the output

Now that we've run MetSim in a notebook we can look at how you might interact with the output in a more meaningful way.
Because we are using an unstructured mesh for the domain we will have to use a bit fancier of techniques than the basic `xarray` commands provide.
Since this is a workshop and we've got a real live audience it might be nice to build something interactive.
We will start by building a map with a time slider that let's us see the values of the output at various times.
Before doing so, however, let's take a brief detour to explain _dictionary comprehensions_.

## Dictionary comprehensions

Dictionary comprehensions are a Python built in functionality which are shorthand for writing for loops.
They, along with _list comprehensions_ are very powerful and concise ways to write code.
We provide a simple example of how they work below.
Following that, we will define a function which will take a variable name and timestep and will produce a map of the variable at that timestep from our MetSim output.

In [ ]:
alphabet = 'abcdefg'

# First, a standard for filling up a dictionary:
index_of_letter = {}
for i in range(len(alphabet)):
    index_of_letter[alphabet[i]] = i
    
# Versus the dictionary comprehension
index2 = {alphabet[i]: i for i in range(len(alphabet))}

print(index_of_letter)
print(index2)

# A mapping function for our output
Below we define the plotting function that we will use to build our interactive plots.
It is heavily commented to make things clear, but the basic ideas is we will take a specific variable at a specific time and plot it using our shapefile for the domain.

In [ ]:
def var_map(var, timestep):
    """
    Create a map for a variable at a given timestep
    Warning: This function is state dependent! 
    
    Params:
        var: the variable of interest
        timestep: the timestep to plot at
    
    Returns:
        poly: A geoviews "shape" plot with the desired data
    """
    # Convert the data from an xarray dataset to a pandas dataframe 
    # This is necessary for geoviews to be able to plot it
    out_df = out[var].sel(time=timestep).to_dataframe()
    # Make sure we have some metadata to join with the shapefile
    out_df['HUC_12'] = gdf['HUC_12'].values
    out_df['hru_name'] = gdf['FIRST_HU_3'].values
    # Create the shape plot - some keys:
    #  - shapes.records() provides the geometry from the shapefiles
    #  - out_df provides the data
    #  - value=var chooses which column of the dataframe to use as data for coloring
    #  - index=['hru_name'] will provide a name on mouse hover
    #  - on='HUC_12' is the key to join the `shapes` with `out_df`
    poly = gv.Shape.from_records(shapes.records(), out_df, value=var, index=['hru_name'], on='HUC_12')
    # Add some options to make things a bit nicer
    #  - width=700, sets the width, as we expect
    #  - cmap='plasma' sets the colormap to 'plasma'
    #  - tools=['hover'] provides information on mouse hover over
    #  - colorbar=True adds a colorbar
    #  - alpha=0.7 adds a bit of transparency
    poly = poly.opts(width=700, cmap='plasma', tools=['hover'], colorbar=True, alpha=0.7)
    return poly

## WARNING: Occasionally these plots don't like to render - try rerunning a few times and they should show up

## Let's try it!

In [ ]:
var_map('temp', out.time.values[3])

## Okay, that was nice, but not very explorable...

We can do one better by making use of the dictionary comprehensions we just discussed.
First we construct a dictionary containing timestampes as keys and maps from the previous plot function as values.
HoloViews provides a nice `HoloMap` function which will then take this mapping and let us interact with the whole thing via a slider.
Note here that the `Map` in `HoloMap` is `Map` [in the programming sense](https://en.wikipedia.org/wiki/Map_(higher-order_function)), not the cartographic sense.

In [ ]:
mapping = {t: var_map('temp', t) for t in out.time.values[0:12]}
hv.HoloMap(mapping, kdims=['timestep'])

## One more layer

We can continue to improve this by adding a timeseries to the side, which will show the mean of the entire domain along with the minimum and maximum for a given variable.
This will let us quickly explore the output dataset and see what's going on in both space and time.

In [ ]:
def interactive_plot(var, start, stop, step):
    # Just as before, use the `var_map` function to build our map
    poly = hv.HoloMap({t: var_map(var, t) for t in out.time.values[start:stop:step]}, kdims=['timestep'])
    # Vlines will give us an indicator of which time slice we are looking at
    vlines = hv.HoloMap({t: hv.VLine(t) for t in out.time.values[start:stop:step]}, kdims=['timestep'])
    # This seems to make the plot appear more often - probably a holo/geoviews bug somewhere 
    poly
    
    # Calculate min, max, and mean over the domain for each time
    vmin = out[var].isel(time=slice(start, stop)).min(dim='hru').rolling(time=step).min().values
    vmax = out[var].isel(time=slice(start, stop)).max(dim='hru').rolling(time=step).max().values
    vmean = out[var].isel(time=slice(start, stop)).mean('hru').rolling(center=True, time=step).mean().dropna('time')
   
    # Build the complete interactive plot. Layers are as follows
    #  - gv.tile_sources.EsriTerrain- Add a background with topography
    #  - * poly - overlay our map onto the background
    #  - + hv.Area... - add the area plot to the right
    #  - * hv.Curve... - overlay the mean curve onto the area plot
    #  - * vlines... - overlay the timestep indicator
    return (gv.tile_sources.EsriTerrain
            * poly
            + (hv
               .Area((out.time[start:stop], vmin, vmax), vdims=['vmin', 'vmax'])
               .opts(alpha=0.5, color='gold', line_color=None)
               .redim.label(x='Date', vmin=var.capitalize()))
            * hv.Curve(vmean).opts(color='purple', alpha=0.8)
            * vlines.opts(alpha=0.4, color='red'))

In [ ]:
interactive_plot('temp', start=0, stop=120, step=6)

# Exploring the effects of different options

Great, so now we've run MetSim in our notebook and have gotten a handle on what's going on in the output.
We will now dig in slightly deeper and start playing with some of the options which MetSim provides.
First we will look at different longwave radiation parameterizations, then we will follow that with a sensitivity analysis of some of the parameters.

## Longwave radiation parameterizations
We have implemented 6 different longwave parameterizations in MetSim.
By default we use the Prata parameterization, but let's see what the others look like.

In [ ]:
# Longwave parameterizations to try
lw_types = ['TVA', 'Anderson', 'Brutsaert', 'Idso', 'Satterlund', 'Prata']
lw_vals = {}
for lw in lw_types:
    print(lw)
    params['lw_type'] = lw.lower()
    ms = MetSim(params)
    ms.run()
    ds = ms.open_output().load()
    lw_vals[lw] = ds['longwave']
    ds.close()

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
for name, lw in lw_vals.items():
    lw.isel(hru=2, time=slice(90,130)).plot(label=name, linewidth=2)
plt.legend()
plt.ylabel('Longwave Radiation ($W/m^2$)')
plt.xlabel('Date')

## Sensitivity analysis: Cloudiness parameterization

The jumps in the longwave plot from above are due to a precipitation event, which causes an estimation of cloud cover to affect the estimated radiation.
Let's try varying the `sw_prec_thresh` and `rain_scalar` parameters, which control corrections during periods of precipitation, effectively parameterizing cloud cover.
The `sw_prec_thresh` determines the amount of precipitation that must be exceeded to be considered "cloudy".
By default this is set to 0, so any amount of precipitation scales the shortwave radiation.
It is scaled by the `rain_scalar` factor, which is 0.75 by default.

In [ ]:
params['lw_type'] = 'prata'
sw_vals = {}
spt_vals = [0.0, 12*0.02, 12*0.05]
rs_vals = [0.6, 0.75, 0.9]
for spt in spt_vals:
    for rs in rs_vals:
        params['sw_prec_thresh'] = spt
        params['rain_scalar'] = rs
        ms = MetSim(params)
        ms.run()
        output = ms.open_output().load()
        rs_dict = sw_vals.get(spt, {})
        rs_dict[rs] = output['shortwave']
        sw_vals[spt] = rs_dict
        output.close()

In [ ]:
def param_map(spt, rs, timestep):
    """
    Create a map for a variable at a given timestep
    Warning: This function is state dependent! 
    
    Params:
        spt: the value for sw_prec_thresh
        rs: the value for rain_scalar
        timestep: the timestep to plot at
    
    Returns:
        poly: A geoviews "shape" plot with the desired data
    """
    out_df = sw_vals[spt][rs].sel(time=timestep).to_dataframe()
    out_df['HUC_12'] = gdf['HUC_12'].values
    out_df['hru_name'] = gdf['FIRST_HU_3'].values
    poly = gv.Shape.from_records(shapes.records(), out_df, value='shortwave', index=['hru_name'], on='HUC_12')
    poly = poly.opts(width=700, cmap='plasma', tools=['hover'], colorbar=True, alpha=0.7)
    return poly

In [ ]:
output = ms.open_output().load()
output['prec'].isel(time=slice(90,130)).plot.line(x='time', add_legend=False, figsize=(12, 6));
output.close()

In [ ]:
mapping = {(spt, rs, t): param_map(spt, rs, t) for t in out.time.values[90:130] 
                                               for spt in spt_vals 
                                               for rs in rs_vals}
(gv.tile_sources.EsriTerrain
 * hv.HoloMap(mapping, kdims=['sw_prec_thresh', 'rain_scalar', 'timestep']))